In [26]:
from selenium import webdriver
import csv

In [4]:
search_list = []
#input = pd.read_csv('ten_sample.csv')
with open('ten_sample.csv', 'r') as input:
    for row in input:
        search_list.append(row)
        

In [24]:
search_list.pop(0)

'"First\tLast\tClass"\n'

In [27]:
driver = webdriver.Chrome()

In [28]:
url = 'https://www.linkedin.com/'
username = 'hbb3xe@virginia.edu'
password = 'S+zMKr5o'

In [29]:
driver.get(url)

In [30]:
driver.find_element_by_id('login-email').send_keys(username)
driver.find_element_by_id('login-password').send_keys(password)

In [31]:
driver.find_element_by_name('submit').click()

In [32]:
driver.find_element_by_id('advanced-search').click()

In [34]:
first = 'Sutton'
last = 'White'
school = 'University of Virginia'
year = '2016'

In [35]:
driver.find_element_by_id('advs-firstName').send_keys(first)
driver.find_element_by_id('advs-lastName').send_keys(last)
driver.find_element_by_id('advs-school').send_keys(school)

In [36]:
driver.find_element_by_class_name('submit-advs').click()

In [38]:
results_summary = driver.find_element_by_id('results-container').text.split('\n')

In [40]:
r_name = results_summary[0].strip('1st')
r_position = results_summary[1]

'Incoming Account Executive at Fisher Investments'

In [44]:
num_results = driver.find_element_by_class_name('search-info').text.split('\n')[0]

In [47]:
num_results = num_results.strip(" results")

In [53]:
num_results = int(num_results)

In [55]:
if num_results is 1:
    print('HI')

HI


'1'

In [56]:
driver.find_element_by_css_selector('a.title.main-headline').click()

In [57]:
work_experience = driver.find_element_by_class_name('background-section').text

In [59]:
work_experience = driver.find_element_by_id('background-experience-container').text

In [60]:
work_experience

'Experience\nAccount Executive\nFisher Investments\nFebruary 2016 – Present (5 months)Camas, Washington\nI will begin this position in July after graduation.\nInvestment Analysis Extern\nRedmond Asset Management, LLC\nJune 2015 – August 2015 (3 months)Richmond, Virginia\n-Researched and presented potential growth and value investment additions to the Small Cap Core Growth composite, the GARP composite, and the All Cap, Equity Income and Global Strategies composite, amounting to roughly $255 million in total assets under management\n\n-Analyzed annual reports, financial statements, SEC 10K filings, and domestic and international financial markets\nInvestment Banking Intern\nDickinson Williams & Company\nJuly 2015 – July 2015 (1 month)Richmond, Virginia Area\n-Used S&P Capital IQ, Microsoft Excel, and data from the U.S. Bureau of Economic Analysis and the St. Louis Federal Reserve to compose financial reports and spreadsheets describing economic trends, capital markets, and M&A activity\

In [25]:
for row in search_list:
    first, last, year = row.split('\t')
    first = first.strip("\"")
    last = last.strip("\"\n")
    print("Processing: {} {} {}".format(first, last, year))
    

Processing: Sutton White 2016"

Processing: Kevin Hsu 2014"

Processing: Ryan Ho 2014"

Processing: Devon Holland 2014"

Processing: Ajay Sundar 2014"

Processing: Graham Campbell 2014"

Processing: David Plon 2011"

Processing: Peter Lizza 2014"

Processing: Alex Sullivan 2014"


In [12]:
a, b, c = first.split('\t')

In [15]:
c

'Class"\n'

In [16]:
a

'"First'

In [20]:
a.strip("\"")

'First'

In [22]:
b

'Last'